# 機械学習フロー
Kaggleの Home Credit Default Risk コンペティションを題材に、機械学習の実践的な流れを学びます。特に適切な 検証 を行い、高い 汎化性能 のあるモデルを完成させることを目指します。

## 【問題1】クロスバリデーション
事前学習期間では検証データをはじめに分割しておき、それに対して指標値を計算することで検証を行っていました。（ホールドアウト法）しかし、分割の仕方により精度は変化します。実践的には クロスバリデーション（交差検証） を行います。分割を複数回行い、それぞれに対して学習と検証を行う方法です。複数回の分割のためにscikit-learnにはKFoldクラスが用意されています。

事前学習期間の課題で作成したベースラインモデルに対してKFoldクラスによるクロスバリデーションを行うコードを作成し実行してください。

sklearn.model_selection.KFold — scikit-learn 0.21.3 documentation

In [1]:
# 自分のマイドライブにマウントする

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
# カレントディレクトリの変更

import os

os.chdir('/content/drive/My Drive/')

# カレントディレクトリの取得
print(os.getcwd())

/content/drive/My Drive


In [3]:
import time

In [4]:
# メモリ使用量の節約関数
import numpy as np
import pandas as pd

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [5]:
train_path = '/content/drive/My Drive/dataset/HomeCredit/dataset/application_train.csv'
test_path = '/content/drive/My Drive/dataset/HomeCredit/dataset/application_test.csv'

In [6]:
#データの読み込み
print("train")
train_raw = import_data(train_path)
print("test")
test_raw = import_data(test_path)

np.set_printoptions(suppress=True)
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 300)

train
Memory usage of dataframe is 286.23 MB
Memory usage after optimization is: 59.54 MB
Decreased by 79.2%
test
Memory usage of dataframe is 45.00 MB
Memory usage after optimization is: 9.40 MB
Decreased by 79.1%


### データの前処理

In [7]:
# 要素数を把握
ntrain = train_raw.shape[0]
ntest = test_raw.shape[0]

# 目的変数をyに格納
target = train_raw['TARGET']
# SK_ID_CURRをX_idに格納
sk_id = test_raw['SK_ID_CURR']

# trainとtestを結合
all_data = pd.concat([train_raw, test_raw]).reset_index(drop=True)
all_data.drop(['SK_ID_CURR', 'TARGET'], axis=1, inplace=True)

In [8]:
# 欠損率の確認
def get_miss(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)

    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

    mis_val_table = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'}
    )

    # 表を欠損率順に並べ替え、欠損値０のサンプルは除外
    mis_val_table_sort = mis_val_table[
        mis_val_table.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1)
    
    return mis_val_table_sort

miss_data = get_miss(all_data)

print("欠損値を含む列数：{}".format(miss_data.shape[0]))
print("-----------------")
print(miss_data)

欠損値を含む列数：67
-----------------
                              Missing Values  % of Total Values
COMMONAREA_MEDI                       248360               69.7
COMMONAREA_AVG                        248360               69.7
COMMONAREA_MODE                       248360               69.7
NONLIVINGAPARTMENTS_MEDI              246861               69.3
NONLIVINGAPARTMENTS_MODE              246861               69.3
NONLIVINGAPARTMENTS_AVG               246861               69.3
FONDKAPREMONT_MODE                    243092               68.2
LIVINGAPARTMENTS_MODE                 242979               68.2
LIVINGAPARTMENTS_MEDI                 242979               68.2
LIVINGAPARTMENTS_AVG                  242979               68.2
FLOORSMIN_MODE                        241108               67.7
FLOORSMIN_MEDI                        241108               67.7
FLOORSMIN_AVG                         241108               67.7
YEARS_BUILD_MODE                      236306               66.3
YEARS_BUIL

In [9]:
# 補完する変数を特定
features = ['EXT_SOURCE_1','EXT_SOURCE_2', 'EXT_SOURCE_3','NAME_TYPE_SUITE',
            'DEF_30_CNT_SOCIAL_CIRCLE','OBS_60_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE',
            'OBS_30_CNT_SOCIAL_CIRCLE','AMT_GOODS_PRICE','AMT_ANNUITY','CNT_FAM_MEMBERS',
            'DAYS_LAST_PHONE_CHANGE']

# カテゴリー変数を特定
categorical = all_data[features].select_dtypes(exclude='number')
# 数値変数を特定
numerical = all_data[features].select_dtypes(include='number')

# カテゴリー変数を最頻値で補完
for f in categorical:
    all_data[f].fillna(all_data[f].mode()[0], inplace=True)

# # 数値変数を中央値で補完
for f in numerical:
    all_data[f].fillna(all_data[f].median(), inplace=True)    

    
# 欠損値10%以上の列を削除
all_data_drop = all_data.dropna(axis=1, thresh=350000)

# データを確認
miss_data = get_miss(all_data_drop)

print("欠損値を含む列数：{}".format(miss_data.shape[0]))
print("-----------------")
print(miss_data)

欠損値を含む列数：0
-----------------
Empty DataFrame
Columns: [Missing Values, % of Total Values]
Index: []


LabelEncoding

In [10]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in all_data_drop:
    if all_data_drop[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(all_data_drop[col].unique())) <= 2:
            # Fit the data
            le.fit(all_data_drop[col])
            # Transform both all_data
            all_data_drop[col] = le.transform(all_data_drop[col])
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('{} columns were label encoded.'.format(le_count))

# One-Hot Encodingを行う
# ソースコードはKaggleのkernelから取得

# one-hot encoding of categorical variables
all_data_lab = pd.get_dummies(all_data_drop)

print('Data shape: ', all_data_lab.shape)

0 columns were label encoded.
Data shape:  (356255, 160)


In [11]:
from sklearn.preprocessing import PolynomialFeatures

all_data_lab = all_data_lab.copy()

# DTIを追加
all_data_lab['DTI'] = all_data_lab['AMT_CREDIT']/all_data_lab['AMT_INCOME_TOTAL']
# DTI計算の元となった特徴量を削除
all_data_lab.drop(columns=['AMT_CREDIT', 'AMT_INCOME_TOTAL'], inplace=True)

# 多次元特徴量の生成
poly = PolynomialFeatures(include_bias=False)

all_data_poly = poly.fit_transform(
    all_data_lab.loc[:, ['AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DTI', 'EXT_SOURCE_2', 'EXT_SOURCE_3']])
feature_names = poly.get_feature_names(
    input_features = ['AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DTI', 'EXT_SOURCE_2', 'EXT_SOURCE_3'])

# # 多次元特徴量生成の元となった特徴量を削除（重複するため）
all_data_lab.drop(columns=['AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DTI', 'EXT_SOURCE_2', 'EXT_SOURCE_3'], inplace=True)

# # 元のデータフレームと多次元特徴量を結合
all_data_lab = pd.concat((all_data_lab,
                             pd.DataFrame(data=all_data_poly, columns=feature_names)),
                             axis=1)

# trainとtestに分割
train = all_data_lab[:ntrain]
test = all_data_lab[ntrain:]

In [12]:
import numpy as np
from sklearn.model_selection import train_test_split

# DataFrameをndarrayに変換
X = np.array(train)
y = np.array(target)

# データを分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25)

In [13]:
# 標準化
from sklearn.preprocessing import QuantileTransformer

scaler = QuantileTransformer(output_distribution='normal')

X_train_std = scaler.fit_transform(X_train)
X_val_std = scaler.transform(X_val)

検証するパラメータチューニング手法の成績を記録するDFの作成

In [14]:
# 各手法の結果を表に記載
summary = pd.DataFrame(index=['Holdout', 'Kfold', 'SSS'], columns=['Default', 'Grid', 'Random', 'Bayesian'])

summary

,Default,Grid,Random,Bayesian
Holdout,NaN,NaN,NaN,NaN
Kfold,NaN,NaN,NaN,NaN
SSS,NaN,NaN,NaN,NaN


### ベースラインモデルの学習（ホールドアウト検証）

上記にて生成した抽出特徴量に対して学習を行う。以後はこのモデルをベースラインと呼ぶ。

分類器：Logistic Regression (max iterations = 5000)
標準化手法：Quantile Transformation (Gaussian Dist.)

In [15]:
# LogisticRegressionベースモデルの正解率を検証

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

lr = LogisticRegression(max_iter=5000)
lr.fit(X_train_std, y_train)

# 特徴量の重要性を取得
features_names = list(train.columns)

feature_importance_values = abs(lr.coef_.flatten())
feature_importances = pd.DataFrame({'feature': features_names, 'importance': feature_importance_values})

lr_pred = lr.predict_proba(X_val_std)

score = roc_auc_score(y_val, lr_pred[:,1])

print("ベースモデルのAUC ROC値：{:5f}".format(score))

ベースモデルのAUC ROC値：0.741684


In [16]:
# 特徴量を重要性の高い順に表示
feature_importances.sort_values('importance', ascending = False, ignore_index=True).head(10)

,feature,importance
0,AMT_GOODS_PRICE^2,1.601562
1,AMT_ANNUITY AMT_GOODS_PRICE,0.923330
2,AMT_ANNUITY DTI,0.868605
3,AMT_GOODS_PRICE DTI,0.817521
4,AMT_ANNUITY EXT_SOURCE_3,0.642810
5,AMT_GOODS_PRICE EXT_SOURCE_3,0.599065
6,DTI^2,0.427998
7,AMT_GOODS_PRICE EXT_SOURCE_2,0.406794
8,NAME_INCOME_TYPE_Maternity leave,0.393480
9,EXT_SOURCE_2,0.350962


In [17]:
#　重要度上位10位までの特徴量を抽出
features_extract = feature_importances[feature_importances['importance'] >= 0.28]

train_ext = train[features_extract['feature']]
test_ext = test[features_extract['feature']]

train_ext

,FLAG_DOCUMENT_2,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Student,EXT_SOURCE_2,AMT_ANNUITY^2,AMT_ANNUITY AMT_GOODS_PRICE,AMT_ANNUITY DTI,AMT_ANNUITY EXT_SOURCE_3,AMT_GOODS_PRICE^2,AMT_GOODS_PRICE DTI,AMT_GOODS_PRICE EXT_SOURCE_2,AMT_GOODS_PRICE EXT_SOURCE_3,DTI^2
0,0,0,0,0.262939,6.101147e+08,8.669875e+09,49595.855469,3443.355957,1.232010e+11,7.047689e+05,92291.750000,48930.906250,4.031618
1,0,0,0,0.622070,1.274383e+09,4.032145e+10,171022.593750,19051.982422,1.275770e+12,5.411152e+06,702628.437500,602804.437500,22.951286
2,0,0,0,0.556152,4.556250e+07,9.112500e+08,13500.000000,4924.072266,1.822500e+10,2.700000e+05,75080.562500,98481.445312,4.000000
3,0,0,0,0.650391,8.812883e+08,8.816891e+09,68758.882812,15843.429688,8.820900e+10,6.879015e+05,193166.015625,158506.343750,5.364628
4,0,0,0,0.322754,4.781001e+08,1.121700e+10,92321.000000,11669.429688,2.631690e+11,2.166000e+06,165572.750000,273783.687500,17.827162
...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,0,0,0,0.681641,7.594434e+08,6.200550e+09,44565.222656,14707.467773,5.062500e+10,3.638572e+05,153369.140625,120080.562500,2.615151
307507,0,0,0,0.115967,1.440360e+08,2.700337e+09,44930.617188,6405.097656,5.062500e+10,8.423438e+05,26092.529297,120080.562500,14.015665
307508,0,0,0,0.535645,8.987404e+08,1.753771e+10,132782.281250,6561.565918,3.422250e+11,2.591068e+06,313352.062500,128040.164062,19.617603
307509,0,0,0,0.514160,4.082420e+08,6.455498e+09,43731.062500,13358.188477,1.020802e+11,6.915157e+05,164274.171875,211231.937500,4.684491


In [18]:
# DataFrameをndarrayに変換
X = np.array(train_ext)
y = np.array(target)

# ホールドアウト法を関数化
def holdout_validate(model, test_size=0.25):
    # データを分割
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size)
    
    scaler = QuantileTransformer(output_distribution='normal')

    # 標準化
    X_train_std = scaler.fit_transform(X_train)
    X_val_std = scaler.transform(X_val)
    
    # 学習・推定
    model.fit(X_train_std, y_train)
    pred = model.predict_proba(X_val_std)
    
    return roc_auc_score(y_val, pred[:,1])

In [19]:
lr = LogisticRegression(max_iter=5000)

score = holdout_validate(lr)

summary['Default']['Holdout'] = score

print("特徴量を抽出したベースモデルのAUC ROC値：{:5f}".format(score))

特徴量を抽出したベースモデルのAUC ROC値：0.716061


### KFoldクロスバリデーション

In [20]:
from sklearn.model_selection import KFold

# KFoldを実施
kf = KFold()

# 分割されたデータのインデックスを表示
for train, val in kf.split(X):
    print("TRAIN SET: {}\n Val SET: {}".format(train, val))

TRAIN SET: [ 61503  61504  61505 ... 307508 307509 307510]
 Val SET: [    0     1     2 ... 61500 61501 61502]
TRAIN SET: [     0      1      2 ... 307508 307509 307510]
 Val SET: [ 61503  61504  61505 ... 123002 123003 123004]
TRAIN SET: [     0      1      2 ... 307508 307509 307510]
 Val SET: [123005 123006 123007 ... 184504 184505 184506]
TRAIN SET: [     0      1      2 ... 307508 307509 307510]
 Val SET: [184507 184508 184509 ... 246006 246007 246008]
TRAIN SET: [     0      1      2 ... 246006 246007 246008]
 Val SET: [246009 246010 246011 ... 307508 307509 307510]


In [21]:
# KFold分割されたデータをを予測する関数を作成
def cross_validate(model, split_size=4):
    results = []
    kf = KFold(n_splits=split_size)
    
    for train_id, val_id in kf.split(X, y):
        train_X = X[train_id]
        train_y = y[train_id]
        val_X = X[val_id]
        val_y = y[val_id]
        
        # 標準化
        scaler = QuantileTransformer(output_distribution='normal')

        train_std = scaler.fit_transform(train_X)
        val_std = scaler.transform(val_X)

        # 学習、推定
        model.fit(train_std, train_y)
        pred = model.predict_proba(val_std)
        
        results.append(roc_auc_score(val_y, pred[:,1]))
    return np.array(results)

In [22]:
start = time.time()

result = cross_validate(lr).mean()

end = time.time()

print("Logistic Regressionの交差検証平均：{:5f}".format(result))
print("学習・推定にかかった時間：{:3f} sec".format(end - start))

Logistic Regressionの交差検証平均：0.712738
学習・推定にかかった時間：54.585473 sec


In [23]:
# cross_val_scoreを使用
# 標準化を行うため、Pipelineを併用

from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

pipeline_lr = Pipeline([('scaler', QuantileTransformer(output_distribution='normal')),
                        ('clf', LogisticRegression(max_iter=5000))])

start = time.time()

score = cross_val_score(pipeline_lr, X, y, scoring='roc_auc', cv=4)

end = time.time()

summary['Default']['Kfold'] = score.mean()

print("LogisticRegressionの交差検証結果：{}".format(score))
print("平均：{:5f}".format(score.mean()))
print("学習・推定にかかった時間：{:3f} sec".format(end - start))

LogisticRegressionの交差検証結果：[0.71500453 0.70949175 0.71331586 0.71323369]
平均：0.712761
学習・推定にかかった時間：48.714142 sec


## 【問題2】グリッドサーチ
これまで分類器のパラメータには触れず、デフォルトの設定を使用していました。パラメータの詳細は今後のSprintで学んでいくことになります。機械学習の前提として、パラメータは状況に応じて最適なものを選ぶ必要があります。最適なパラメータを探していくことをパラメータチューニングと呼びます。パラメータチューニングをある程度自動化する単純な方法としてはグリッドサーチがあります。

scikit-learnのGridSearchCVを使い、グリッドサーチを行うコードを作成してください。そして、ベースラインモデルに対して何らかしらのパラメータチューニングを行なってください。どのパラメータをチューニングするかは、使用した手法の公式ドキュメントを参考にしてください。

sklearn.model_selection.GridSearchCV — scikit-learn 0.21.3 documentation

GridSearchCVクラスには引数としてモデル、探索範囲、さらにクロスバリデーションを何分割で行うかを与えます。クロスバリデーションの機能も含まれているため、これを使用する場合はKFoldクラスを利用する必要はありません。

In [24]:
# LogisticRegressionのハイパーパラメータチューニング
from sklearn.model_selection import GridSearchCV

pipeline_lr = Pipeline([('scaler', QuantileTransformer(output_distribution='normal')),
                        ('clf', LogisticRegression(max_iter=5000))])

# LogisticRegressionのparameter gridを設定
param_grid = { 
    'clf__penalty': ['none', 'l2'],
    'clf__C': [10**i for i in range(-4,4)]
}

# Grid search実施
start = time.time()
grid_search = GridSearchCV(pipeline_lr, param_grid, cv=4, scoring='roc_auc')
grid_search.fit(X, y)
end = time.time()

print('Best parameters: {}'.format(grid_search.best_params_))
print('Best score: {}'.format(grid_search.best_score_))
print("探索・学習・推定にかかった時間：{:3f} sec".format(end - start))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and

Best parameters: {'clf__C': 1000, 'clf__penalty': 'l2'}
Best score: 0.7127623089420049
探索・学習・推定にかかった時間：603.409720 sec


In [25]:
# grid searchで得られたハイパーパラメータを設定
lr_grid = LogisticRegression(C=0.01, penalty='none', max_iter=5000)

pipe_lr_grid = Pipeline([('scaler', QuantileTransformer(output_distribution='normal')),
                          ('clf', lr_grid)])

score = cross_val_score(pipe_lr_grid, X, y, scoring='roc_auc', cv=4).mean()

summary['Grid']['Kfold'] = score

print("チューニング後のvalidation ROC AUC値：{:5f}".format(score))

pipe_lr_grid.fit(X_train, y_train)
pipe_lr_grid_pred = pipe_lr_grid.predict_proba(X_val)

print("検証用データに対するチューニング後のROC AUC値：{:5f}".
      format(roc_auc_score(y_val, pipe_lr_grid_pred[:,1])))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "


チューニング後のvalidation ROC AUC値：0.712644


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "


検証用データに対するチューニング後のROC AUC値：0.741952


In [26]:
# ホールドアウト法のスコアも検証
score = holdout_validate(lr_grid)
summary['Grid']['Holdout'] = score
print("チューニング後のholdout ROC AUC値：{:5f}".format(score))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "


チューニング後のholdout ROC AUC値：0.711559


## 【問題3】Kaggle Notebooksからの調査
KaggleのNotebooksから様々なアイデアを見つけ出して、列挙してください。

- Logistic Regressionのハイパーパラメータを調査
- 層化シャッフル分割交差検証を実装
- random search実装
- bayesian optimization実装
- 別の分類器を試す


Logistic Regressionの仕組みを改めて理解する

[Qiita - ロジスティック回帰の初歩を理解する](https://qiita.com/iwashi-kun/items/030ed4dc48b16ca1b5a1)

パラメータについて調査

[Don’t Sweat the Solver Stuff](https://towardsdatascience.com/dont-sweat-the-solver-stuff-aea7cddc3451)

[Discussion - Stack Overflow](https://stackoverflow.com/questions/38640109/logistic-regression-python-solvers-defintions/52388406#52388406)

様々なcross validation手法について

[交差検証（cross validation／クロスバリデーション）の種類を整理してみた](https://aizine.ai/cross-validation0910/#toc9)

## 【問題4】高い汎化性能のモデル作成¶
問題3で見つけたアイデアと、独自のアイデアを組み合わせ高い汎化性能のモデル作りを進めてください。

その過程として、何を行うことで、クロスバリデーションの結果がどの程度変化したかを表にまとめてください。
<br>
<br>

層化シャッフル分割交差検証
KFoldでの交差検証は時間がかかりすぎることがわかった。

また、このHome Creditデータはクラスが不均衡なため、別の交差検証手法を採用する。
時間短縮を図ったShuffleSplit、および不均衡に対応するためのStratifiedKFoldを合わせたStratifiedShuffleSplitを採用する。

In [27]:
# 層化シャッフル分割交差検証を読み込む
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit()

# 分割されたデータのインデックスを表示
for train, val in sss.split(X, y):
    print("TRAIN sample size: {}\n VAL sample size: {}".format(len(train), len(val)))

TRAIN sample size: 276759
 VAL sample size: 30752
TRAIN sample size: 276759
 VAL sample size: 30752
TRAIN sample size: 276759
 VAL sample size: 30752
TRAIN sample size: 276759
 VAL sample size: 30752
TRAIN sample size: 276759
 VAL sample size: 30752
TRAIN sample size: 276759
 VAL sample size: 30752
TRAIN sample size: 276759
 VAL sample size: 30752
TRAIN sample size: 276759
 VAL sample size: 30752
TRAIN sample size: 276759
 VAL sample size: 30752
TRAIN sample size: 276759
 VAL sample size: 30752


In [28]:
# 層化シャッフル分割分割されたデータを検証する関数を作成

def sss_cross_validate(model, split_size=4):
    results = []
    sss = StratifiedShuffleSplit(n_splits=split_size)
    
    for train_id, val_id in sss.split(X, y):
        train_X = X[train_id]
        train_y = y[train_id]
        val_X = X[val_id]
        val_y = y[val_id]
        
        # 標準化
        scaler = QuantileTransformer(output_distribution='normal')

        train_std = scaler.fit_transform(train_X)
        val_std = scaler.transform(val_X)
        
        # 学習、推定
        model.fit(train_std, train_y)
        pred = model.predict_proba(val_std)
        
        results.append(roc_auc_score(val_y, pred[:,1]))
    return np.array(results)

In [29]:
# 層化シャッフル分割のスコアを検証
start = time.time()

score = sss_cross_validate(lr_grid).mean()

end = time.time()

summary['Grid']['SSS'] = score

print("Logistic Regressionの層化シャッフル分割交差検証平均：{:5f}".format(score))
print("学習・推定にかかった時間：{:3f} sec".format(end - start))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "


Logistic Regressionの層化シャッフル分割交差検証平均：0.712028
学習・推定にかかった時間：57.793517 sec


In [30]:
# デフォルトパラメータの場合も検証
score = sss_cross_validate(lr).mean()

summary['Default']['SSS'] = score
print("デフォルトパラメータ＋層化シャッフル分割交差検証平均：{:5f}".format(score))

デフォルトパラメータ＋層化シャッフル分割交差検証平均：0.714270


#### RandomSearch

In [31]:
# Random Searchでハイパーパラメータを探索
from sklearn.model_selection import RandomizedSearchCV

param_grid = { 
    'clf__penalty': ['l1', 'l2'],
    'clf__C': [10**i for i in range(-4,4)],
    'clf__solver': ['lbfgs', 'liblinear', 'saga']
}


start = time.time()
rand_search = RandomizedSearchCV(pipeline_lr, param_grid, n_iter=10, cv=4,
                                 n_jobs=-1, scoring='roc_auc')
rand_search.fit(X, y)
end = time.time()

print('Best parameters: {}'.format(rand_search.best_params_))
print("探索・学習・推定にかかった時間：{:3f} sec".format(end - start))

Best parameters: {'clf__solver': 'liblinear', 'clf__penalty': 'l1', 'clf__C': 1}
探索・学習・推定にかかった時間：740.656221 sec


In [32]:
# RandomizedSearchのスコア
lr_rand = LogisticRegression(solver='liblinear', penalty='l1', C=100, max_iter=5000)
pipe_lr_rand = Pipeline([('scaler', QuantileTransformer(output_distribution='normal')),
                          ('clf', lr_rand)])

score_randh = holdout_validate(lr_rand)
summary['Random']['Holdout'] = score_randh
print("Random Searchパラメータ＋ホールドアウト分割交差検証平均：{:5f}".format(score_randh))

score_rands = sss_cross_validate(lr_rand).mean()
summary['Random']['SSS'] = score_rands
print("Random Searchパラメータ＋層化シャッフル分割交差検証平均：{:5f}".format(score_rands))

score_randk = cross_val_score(pipe_lr_rand, X, y, scoring='roc_auc', cv=4).mean()
summary['Random']['Kfold'] = score_randk
print("Random Searchパラメータ＋KFold分割交差検証平均：{:5f}".format(score_randk))

Random Searchパラメータ＋ホールドアウト分割交差検証平均：0.716909
Random Searchパラメータ＋層化シャッフル分割交差検証平均：0.710748
Random Searchパラメータ＋KFold分割交差検証平均：0.712716


#### Bayesian Optimization

In [33]:
# hyperopt実装
from hyperopt import hp, tpe, Trials, fmin

hyperopt_params = {
    'C': hp.loguniform('C', -4, 4),
    'warm_start': hp.choice('warm_start', [True, False]),
    'solver': hp.choice('solver', ['liblinear', 'lbfgs', 'saga']),
    'fit_intercept': hp.choice('fit_intercept', [True, False]),
    'class_weight': hp.choice('class_weight', [None, 'balanced'])
}


def objective(params, X=X, y=y, cv=4):
    # モデルのインスタンス化
    classifier = LogisticRegression(**params, max_iter=5000)
    # cross validationを計算
    scores = cross_validate(classifier)
    # scoresの平均値を取得
    score = scores.mean()
    # roc aucを最大化したいので、1から引いた値を出力
    loss = 1 - score
    return loss


trials = Trials()
n_iter = 50

start = time.time()

best = fmin(fn=objective,
          space=hyperopt_params, 
          algo=tpe.suggest,
          max_evals=n_iter, 
          trials=trials,
          verbose=1
         )

end = time.time()

print("Best params {}".format(best))
print("探索にかかった時間：{:3f} sec".format(end - start))

  6%|▌         | 3/50 [14:01<3:20:25, 255.86s/it, best loss: 0.2867712473980111]

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)



100%|██████████| 50/50 [42:29<00:00, 50.99s/it, best loss: 0.28660577762610073]
Best params {'C': 20.563298284262707, 'class_weight': 1, 'fit_intercept': 1, 'solver': 2, 'warm_start': 0}
探索にかかった時間：2549.296064 sec


In [34]:
# Bayesian Optimizationで得られたハイパーパラメータを設定
lr_bayes = LogisticRegression(C=1.650775575860084, warm_start=False, fit_intercept=True, class_weight='balanced', solver='lbfgs', max_iter=5000)

pipe_lr_bayes = Pipeline([('scaler', QuantileTransformer(output_distribution='normal')),
                          ('clf', lr_bayes)])

score_bayes_h = holdout_validate(lr_bayes)
summary['Bayesian']['Holdout'] = score_bayes_h
print("Bayesian Searchパラメータ＋ホールドアウト分割交差検証平均：{:5f}".format(score_bayes_h))

score_bayes_k = cross_val_score(pipe_lr_bayes, X, y, scoring='roc_auc', cv=4).mean()
summary['Bayesian']['Kfold'] = score_bayes_k
print("Bayesian Searchパラメータ＋KFold分割交差検証平均：{:5f}".format(score_bayes_k))

score_bayes_s = sss_cross_validate(lr_bayes).mean()
summary['Bayesian']['SSS'] = score_bayes_s
print("Bayesian Searchパラメータ＋層化シャッフル分割交差検証平均：{:5f}".format(score_bayes_s))

Bayesian Searchパラメータ＋ホールドアウト分割交差検証平均：0.712186
Bayesian Searchパラメータ＋KFold分割交差検証平均：0.713248
Bayesian Searchパラメータ＋層化シャッフル分割交差検証平均：0.717821


### 各手法の成績確認

In [35]:
# 各手法の平均スコアを表示
summary

,Default,Grid,Random,Bayesian
Holdout,0.716061,0.711559,0.716909,0.712186
Kfold,0.712761,0.712644,0.712716,0.713248
SSS,0.71427,0.712028,0.710748,0.717821


### 分類器にLightGBMを使用してみる

In [36]:
import lightgbm as lgb

X = train_ext
y = target

train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.25)

lgb_train = lgb.Dataset(data=train_X, label=train_y)
lgb_val = lgb.Dataset(data=val_X, label=val_y, reference=lgb_train)

In [37]:
params = {'task': 'train', 'boosting_type': 'gbdt', 
          'objective': 'binary', 'metric': 'auc', 'n_estimators':5000}

print("ベースラインlightGBMのHoldout検証")
model = lgb.train(params, lgb_train, valid_sets=lgb_val,
                  early_stopping_rounds=150, verbose_eval=200)

ベースラインlightGBMのHoldout検証


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 150 rounds.
[200]	valid_0's auc: 0.723427
Early stopping, best iteration is:
[114]	valid_0's auc: 0.724536


In [38]:
lgb_train_cv = lgb.Dataset(data=X, label=y)
scores = lgb.cv(params, lgb_train_cv, num_boost_round=5000,
                    early_stopping_rounds=150, nfold=4)

print("ベースラインlightGBMの交差検証ベストスコア：{:5f}".format(max(scores['auc-mean'])))

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:430: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


ベースラインlightGBMの交差検証ベストスコア：0.722747


In [39]:
from hyperopt import STATUS_OK

lgb_params = {
    'boosting_type': hp.choice('boosting_type', 
                [{'boosting_type': 'gbdt', 'subsample': hp.uniform('gdbt_subsample', 0.5, 1)}, 
#                  {'boosting_type': 'dart', 'subsample': hp.uniform('dart_subsample', 0.5, 1)},
                 {'boosting_type': 'goss', 'subsample': 1.0}
                ]),
    'class_weight': hp.choice('class_weight', [None, 'balanced']),
    'num_leaves': hp.quniform('num_leaves', 30, 151, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
    'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0)
}

lgb_train = lgb.Dataset(X, y)

def objective(params, n_folds=4):
    
    # nested spaceの取得
    params['subsample'] = params['boosting_type']['subsample']
    boost_type = params['boosting_type']['boosting_type']
    del params['boosting_type']
    params['boosting_type'] = boost_type
    
    # 数値型への変換
    params['num_leaves'] = int(params['num_leaves'])
    params['min_child_samples'] = int(params['min_child_samples'])
    params['subsample_for_bin'] = int(params['subsample_for_bin'])
    
    # cross validationを計算
    scores = lgb.cv(params, lgb_train, num_boost_round=500,
                    early_stopping_rounds=150, nfold=n_folds, metrics='auc')
    
    # scoresの平均値を取得
    best_score = max(scores['auc-mean'])
    # roc aucを最大化したいので、1から引いた値を出力
    loss = 1 - best_score
    
    return {'loss': loss, 'params': params, 'status': STATUS_OK}


trials = Trials()
n_iter = 100

start = time.time()

best = fmin(fn=objective,
          space=lgb_params, 
          algo=tpe.suggest,
          max_evals=n_iter, 
          trials=trials,
          verbose=1
         )

end = time.time()

print("Best params {}".format(best))
print("探索にかかった時間：{:3f} sec".format(end - start))

100%|██████████| 100/100 [1:06:31<00:00, 39.92s/it, best loss: 0.27474512724937683]
Best params {'boosting_type': 0, 'class_weight': 0, 'colsample_by_tree': 0.6510815541840225, 'gdbt_subsample': 0.6085645934050514, 'learning_rate': 0.02169002549386613, 'min_child_samples': 480.0, 'num_leaves': 30.0, 'reg_alpha': 0.8131635660390554, 'reg_lambda': 0.22981094431865268, 'subsample_for_bin': 40000.0}
探索にかかった時間：3991.935719 sec


In [40]:
# チューニングしたモデルの検証
params = {'task': 'train', 'boosting_type': 'gbdt', 
          'objective': 'binary', 'metric': 'auc', 'n_estimators':5000,
          'class_weight': None, 'colsample_by_tree': 0.6127854826784527, 'gdbt_subsample': 0.9903813589683813,
          'learning_rate': 0.010170991958005731, 'min_child_samples': 485, 'num_leaves': 53, 
          'reg_alpha': 0.9983892325538006, 'reg_lambda': 0.6735746969077508, 'subsample_for_bin': 180000}

lgb_train_cv = lgb.Dataset(data=X, label=y)
scores = lgb.cv(params, lgb_train_cv, num_boost_round=5000,
                    early_stopping_rounds=150, nfold=4)

print("チューニング後lightGBMの交差検証ベストスコア：{:5f}".format(max(scores['auc-mean'])))

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:430: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


チューニング後lightGBMの交差検証ベストスコア：0.724288


## 【問題5】最終的なモデルの選定¶
最終的にこれは良いというモデルを選び、推定した結果をKaggleに提出してスコアを確認してください。どういったアイデアを取り入れ、どの程度のスコアになったかを記載してください。
<br>
<br>

- 最適化手法：Bayesian Optimization
- 分類器：LightGBM


In [41]:
# Submission用DF作成
params = {'task': 'train', 'boosting_type': 'gbdt', 
          'objective': 'binary', 'metric': 'auc', 'n_estimators':5000,
          'class_weight': None, 'colsample_by_tree': 0.6127854826784527, 'gdbt_subsample': 0.9903813589683813,
          'learning_rate': 0.010170991958005731, 'min_child_samples': 485, 'num_leaves': 53, 
          'reg_alpha': 0.9983892325538006, 'reg_lambda': 0.6735746969077508, 'subsample_for_bin': 180000}

X = train_ext
y = target

train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.25, stratify=y)

lgb_train = lgb.Dataset(data=train_X, label=train_y)
lgb_val = lgb.Dataset(data=val_X, label=val_y, reference=lgb_train)

model_submit = lgb.train(params, lgb_train, valid_sets=lgb_val,
                  early_stopping_rounds=150, verbose_eval=200)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 150 rounds.
[200]	valid_0's auc: 0.718971
[400]	valid_0's auc: 0.723363
[600]	valid_0's auc: 0.724303
[800]	valid_0's auc: 0.724498
Early stopping, best iteration is:
[767]	valid_0's auc: 0.724537


In [42]:
pred_submit = model_submit.predict(test_ext)
output = pd.DataFrame({'SK_ID_CURR': sk_id,
                       'TARGET': pred_submit})

output

,SK_ID_CURR,TARGET
0,100001,0.093642
1,100005,0.157781
2,100013,0.027563
3,100028,0.030741
4,100038,0.095400
...,...,...
48739,456221,0.039468
48740,456222,0.041991
48741,456223,0.082866
48742,456224,0.082766


In [43]:
# csvに書き込み
output.to_csv('submission1.csv', index=False)